In [3]:
import os
from PIL import Image
import numpy as np
import cv2

In [4]:
def vectorizar(carpeta, label):
    archivos = os.listdir(carpeta)

    # Lista para almacenar los vectores de las imágenes
    vectores_imagenes = []

    # Iterar sobre los archivos
    for archivo in archivos:
        # Comprobar si es un archivo de imagen
        if archivo.endswith(".jpg") or archivo.endswith(".png"):
            # Ruta completa de la imagen
            ruta_imagen = os.path.join(carpeta, archivo)
            
            # Cargar la imagen
            imagen = Image.open(ruta_imagen)

            # Redimensiona la imagen a 8x8 píxeles
            imagen = imagen.resize((50, 125))

            # Convierte la imagen a escala de grises
            imagen = imagen.convert("L")

            # Convierte la imagen a un array de NumPy
            array_imagen = np.array(imagen)

            # Aplana el array de imagen a un vector de 1x64
            vector = array_imagen.flatten()
            
            # Crea una lista con la etiqueta y el nombre del archivo
            etiqueta_archivo = [label, archivo]

            # Agrega la lista al final del vector
            vector_con_etiqueta_archivo = np.append(vector, etiqueta_archivo)

            # Agrega el vector a la lista de vectores de imágenes
            vectores_imagenes.append(vector_con_etiqueta_archivo)

    # Convierte la lista de vectores en una matriz de NumPy
    return np.array(vectores_imagenes)


In [5]:
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

class KNNClassifier:
    def __init__(self, k):
        self.k = k

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        y_pred = []
        for i in range(len(X_test)):
            distances = []
            for j in range(len(self.X_train)):
                dist = euclidean_distance(X_test[i], self.X_train[j])
                distances.append((dist, self.y_train[j]))
            distances.sort(key=lambda x: x[0])  # Ordenar distancias de menor a mayor
            neighbors = distances[:self.k]  # Obtener los k vecinos más cercanos
            classes = [neighbor[1] for neighbor in neighbors]  # Obtener las clases de los vecinos
            y_pred.append(max(set(classes), key=classes.count))  # Clasificación por voto mayoritario
        return y_pred

In [6]:
def mergeData(data1, data2):
    return np.concatenate((data1, data2), axis=0)

In [7]:
def exclude(matriz, indice):
    filas_seleccionadas = []
    for i, fila in enumerate(matriz):
        if i != indice:
            filas_seleccionadas.append(fila)
    return np.array(filas_seleccionadas)

In [8]:
def LOO_Knn(allData):
    labelsPredictedKNN = []
    labelReal = allData[:, 6250]
    for i in range(allData.shape[0]):
        dataTrain = exclude(allData, i)
        ImgsTrain = np.array(dataTrain[:, :-2], dtype= int)
        LabelsTrain = np.array(dataTrain[:, 6250], dtype= int)
        NameImageTrain = np.array(dataTrain[:, 6251], dtype= str)  


        dataTest = allData[i, :]
        dataTest = dataTest.reshape(1, 6252)
        ImgsTest = np.array(dataTest[:, :-2], dtype=int)
        LabelsTest = np.array(dataTest[:, 6250], dtype=int)
        NameImageTest = np.array(dataTest[:, 6251], dtype=str)

        knn = KNNClassifier(1)
        knn.fit(ImgsTrain, LabelsTrain)
        prediction = knn.predict(ImgsTest)
        
        labelsPredictedKNN.append(prediction[0])
        #print(prediction[0], LabelsTest[0])
    labelsPredictedKNN = np.array(labelsPredictedKNN)  
    return(labelsPredictedKNN)

In [9]:
data_yes = vectorizar('carpeta_MitadesYes/', 1)
data_no = vectorizar('carpeta_MitadesNo/', 0)

In [10]:
allData = mergeData(data_yes, data_no)
print(allData.shape)

(207, 6252)


In [11]:
predictions = LOO_Knn(allData)

In [12]:
print(predictions)
print(np.array(allData[:, 6250], dtype=int))

[0 0 1 1 1 1 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1
 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 0 1 1 0 1 0
 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0
 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [13]:
curacy = np.mean(predictions == np.array(allData[:, 6250], dtype=int))
print(accuracy)

NameError: name 'accuracy' is not defined